In [1]:
import sys
sys.path.append('/data/hanweiguang/Projects/PIXIU')

In [25]:
from src.utils import MultiClient
from sklearn.metrics import confusion_matrix, matthews_corrcoef, f1_score, accuracy_score
import json

In [3]:
GENERATION_CONFIG = [
    0.1,  # int | float (numeric value between 0 and 1) in 'Temperature' Slider component
    0.75,  # int | float (numeric value between 0 and 1) in 'Top p' Slider component
    40,  # int | float (numeric value between 0 and 100) in 'Top k' Slider component
    1,  # int | float (numeric value between 1 and 4) in 'Beams Number' Slider component
    True, # do sample
    8,  # int | float (numeric value between 1 and 2000) in 'Max New Tokens' Slider component
    1,  # int | float (numeric value between 1 and 300) in 'Min New Tokens' Slider component
    1.2,  # int | float (numeric value between 1.0 and 2.0) in 'Repetition Penalty' Slider component
]

In [4]:
with open("/data/hanweiguang/Projects/PIXIU/data/cikm18/test.jsonl") as f:
    data = f.readlines()
    data = [json.loads(val) for val in data]

In [5]:
worker_addrs = [
    f"http://127.0.0.1:{17860 + i}" for i in range(1)
]
clients = MultiClient(worker_addrs)

Loaded as API: http://127.0.0.1:17860/ ✔


In [6]:
results = clients.predict(
    [
        [
            datum["conversations"][0]["value"]
        ] + GENERATION_CONFIG for datum in data[:200]
    ]
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [03:21<00:00,  1.01s/it]


In [11]:
labels = [
    datum["label"] for datum in data[:200]
]

In [12]:
y_true = [1 if i == "Rise" else 0 for i in labels]
y_pred = [1 if i == "Rise" else 0 for i in results]

In [22]:
# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

# Calculate Matthews correlation coefficient
mcc = matthews_corrcoef(y_true, y_pred)
print(f'MCC: {mcc}')

MCC: -0.05380001385625025


In [23]:
f1 = f1_score(y_true, y_pred, average='weighted')
print(f'F1: {f1}')

F1: 0.4573069852941177


In [27]:
accuracy = accuracy_score(y_true, y_pred)
print(f'accuracy: {accuracy}')

accuracy: 0.51
